In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

Classification
========


To visualize the workings of machine learning algorithms, it is often helpful to study two-dimensional or one-dimensional data, that is data with only one or two features. 

While in practice, datasets usually have many more features, it is hard to plot high-dimensional data on two-dimensional screens.

We will illustrate some very simple examples before we move on to more "real world" data sets.

First, we will look at a two class classification problem in two dimensions. We use the synthetic data generated by the ``make_blobs`` function.

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(centers=2, random_state=0)
print(X.shape)
print(y.shape)
print(X[:5, :])
print(y[:5])

As the data is two-dimensional, we can plot each sample as a point in two-dimensional space, with the first feature being the x-axis and the second feature being the y-axis.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")

As classification is a supervised task, and we are interested in how well the model generalizes, we split our data into a training set,
to built the model from, and a test-set, to evaluate how well our model performs on new data. 
The ``train_test_split`` function form the ``cross_validation`` module does that for us, by randomly splitting of 25% of the data for testing.

<img src="images/train_test_split.svg" width="80%">


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=0)

<img src="images/supervised_workflow.svg" width="80%">


## Scikit-Learn Estimator API

Every algorithm is exposed in scikit-learn via an ''Estimator'' object. For instance a logistic regression is:

In [ ]:
from sklearn.linear_model import LogisticRegression

All models in scikit-learn have a very consistent interface.
First, we instantiate the estimator object.

In [ ]:
classifier = LogisticRegression()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

To built the model from our data, that is to learn how to classify new points, we call the ``fit`` function with the training data, and the corresponding training labels (the desired output for the training data point):

In [ ]:
classifier = classifier.fit(X_train, y_train)
classifier

We can then apply the model to unseen data and use the model to predict the estimated outcome using the ``predict`` method:

In [ ]:
prediction = classifier.predict(X_test)

We can compare these against the true labels:

In [ ]:
print(prediction)
print(y_test)

We can evaluate our classifier quantitatively by measuring what fraction of predictions is correct (i.e. **accuracy**):

In [ ]:
np.mean(prediction == y_test)

There is also a convenience function , ``score``, that all scikit-learn classifiers have to compute this directly from the test data:
    

In [ ]:
classifier.score(X_test, y_test)

It is often helpful to compare the generalization performance (on the test set) to the performance on the training set:

In [ ]:
classifier.score(X_train, y_train)

LogisticRegression is a so-called linear model,
that means it will create a decision that is linear in the input space. 

In 2D, this simply means it finds a line to separate the blue from the red:

In [ ]:
def plot_2d_separator(classifier, X, fill=False, ax=None, eps=None):
    if eps is None:
        eps = X.std() / 2.
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 100)
    yy = np.linspace(y_min, y_max, 100)

    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    try:
        decision_values = classifier.decision_function(X_grid)
        levels = [0]
        fill_levels = [decision_values.min(), 0, decision_values.max()]
    except AttributeError:
        # no decision_function
        decision_values = classifier.predict_proba(X_grid)[:, 1]
        levels = [.5]
        fill_levels = [0, .5, 1]

    if ax is None:
        ax = plt.gca()
    if fill:
        ax.contourf(X1, X2, decision_values.reshape(X1.shape),
                    levels=fill_levels, colors=['blue', 'red'])
    else:
        ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels,
                   colors="black")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(classifier, X)

**Estimated parameters**: All the estimated parameters are attributes of the estimator object ending by an underscore. 

Here, these are the coefficients and the offset of the line:

In [ ]:
print(classifier.coef_)
print(classifier.intercept_)

Another classifier: K Nearest Neighbors
------------------------------------------------

Another popular and easy to understand classifier is K nearest neighbors (kNN).  

It has one of the simplest learning strategies: given a new, unknown observation, look up in your reference database which ones have the closest features and assign the predominant class.

The interface is exactly the same as for ``LogisticRegression``.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

This time we set a parameter of the KNeighborsClassifier to tell it we only want to look at one nearest neighbor:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

We fit the model with out training data

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=40)
plt.xlabel("first feature")
plt.ylabel("second feature")
plot_2d_separator(knn, X)

In [ ]:
knn.score(X_test, y_test)

Exercise
=========
Apply the KNeighborsClassifier to the ``iris`` dataset. Play with different values of the ``n_neighbors`` and observe how training and test score change.